In [9]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches_latest.csv").dropna()
df = df[(df.league=="Barclays Premier League") | (df.league=="UEFA Champions League")].reset_index(drop=True)

In [29]:
df["spi_margin"] = np.abs(df["spi1"] - df["spi2"])
df["h_spi"] = df["spi1"] - df["spi2"]

In [30]:
spi = []
m_spim = np.mean(df["spi_margin"])
sd_spim = np.std(df["spi_margin"])

In [31]:
for x in df["spi_margin"]:
    if x < m_spim:
        spi.append("CLOSE MATCHUP")
    elif (x >= m_spim) & (x < (m_spim + sd_spim)):
        spi.append("CLOSE FAVORITE")
    elif (x >= (m_spim + sd_spim)) & (x < (m_spim + (sd_spim * 1.5))):
        spi.append("CLEAR FAVORITE")
    else:
        spi.append("SPI BLOWOUT")

In [32]:
df["spi_bucket"] = spi

In [33]:
X_final = df[['spi1', 'spi2', 'spi_margin', 'h_spi']]
y_final = df[['spi_bucket']].copy()
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=6)

In [42]:
model = XGBClassifier(n_estimators=400, objective="multi:softprob", verbosity=1).fit(X_train, y_train.values.ravel())

[11:27:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [51]:
predictions = model.predict(X_test)

In [52]:
accuracy = accuracy_score(y_test, predictions)

In [53]:
print("Test Accuracy Score: {}%".format(accuracy * 100))

Test Accuracy Score: 98.98989898989899%
